## Similarity Analysis of Two Congress Speeches
### By calculating the Cosine Similarity between the TF-IDF (term frequency-inverse document frequency) and Count Vectors of texts with n-grams
Using two speeches of my choosing,<br />
preparing them for analysis (removing punctuation, stop-words),<br />
using Bag of Words and n-grams in addition to tf-idf to find the cosine similarity between them.<br />
Discussing my findings.

In [1]:
""" 
%pip install -U scikit-learn
%pip install nltk 
"""

' \n%pip install -U scikit-learn\n%pip install nltk \n'

In [2]:
""" 
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
"""

" \nnltk.download('punkt')\nnltk.download('wordnet')\nnltk.download('averaged_perceptron_tagger')\nnltk.download('vader_lexicon')\n"

In [3]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.pipeline import Pipeline

# import nltk delete if not necessary
# import string
# import numpy as np delete if not necessary

In [4]:
import requests

text1_url = "https://raw.githubusercontent.com/ariedamuco/ML-for-NLP/main/Inputs/105-extracted-date/105-moseleybraun-il.txt"
text2_url = "https://raw.githubusercontent.com/ariedamuco/ML-for-NLP/main/Inputs/105-extracted-date/105-reid-nv.txt"

text1_get, text2_get = requests.get(text1_url), requests.get(text2_url)
text1, text2 = text1_get.text, text2_get.text

print("text1 head:\n",text1[0:200],"\n\ntext2 head:\n",text2[0:200])

text1 head:
 <DOC>
<DOCNO>105-moseleybraun-il-1-19981009</DOCNO>
<TEXT>
 Ms. MOSELEYBRAUN. Mr. President, I want to note my disappointment that the permanent relief for Haitian refugees that I and many others in t 

text2 head:
 <DOC>
<DOCNO>105-reid-nv-1-19981020</DOCNO>
<TEXT>
 Mr. REID. Mr. President, I rise today to call attention to the outstanding achievements of a Nevadan who has dedicated himself to helping individual


In [5]:
""" 
# Loading the chosen speech texts:
text1 = open(r"Congress Speeches\105-moseleybraun-il.txt").read()
text2 = open(r"Congress Speeches\105-reid-nv.txt").read()

# Printing the first 200 characters:
print("text1 head:\n",text1[0:200],"\n\ntext2 head:\n",text2[0:200])
"""

' \n# Loading the chosen speech texts:\ntext1 = open(r"Congress SpeechesE-moseleybraun-il.txt").read()\ntext2 = open(r"Congress SpeechesE-reid-nv.txt").read()\n\n# Printing the first 200 characters:\nprint("text1 head:\n",text1[0:200],"\n\ntext2 head:\n",text2[0:200])\n'

##### Removing punctuation and stop-words in the text

In [6]:
def text_preprocessor(text):
    # Deleting non-word characters by replacing them with blank (' '):
    text= re.sub(r'\W',' ', text)
    # Tokenizing the string text into word substrings, writing them to a list (.lower() makes all characters lower case):
    tokens = word_tokenize(text.lower())
    # Removing English stopwords from the list:
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    # Keeping words with at least 3 characters in the list:
    tokens = [word for word in tokens if len(word)>=3]
    # Joining the tokens -substrings- in the list back together with blank (' ') between them:
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text 

In [7]:
# Testing the pre-processing function with the first 1000 characters of the first text:
text1_head_tokenized = text_preprocessor(text1[:1000])
text1_head_tokenized

'doc docno 105 moseleybraun 19981009 docno text moseleybraun president want note disappointment permanent relief haitian refugees many others body worked make law dropped treasury appropriations conference report effort began last year debate appropriations bill included language granted certain central americans access suspension deportation procedure haitians granted access may recall supported granting relief affected class central americans along several colleagues senate house fought vigorously additional provisions haitian refugees although unsuccessful effort later introduced 1504 haitian immigrations fairness act 1997 legislation would provide haitian refugees permanent residency status course'

##### Stemming the words in the tokenized text

In [8]:
def stem_words(text):
    # Creating a stemmer instance which uses Porter Stemming Algorithm:
    stemmer = PorterStemmer()
    # Tokenizing the text into words, stemming them:
    stemmed_words = [stemmer.stem(word) for word in word_tokenize(text)]
    # Joining the word stems back and returning:
    return ' '.join(stemmed_words)

# Some alternatives to Porter in NLTK are Snowball (in English) and Lancaster.

In [9]:
# Testing the stemmer function:
text1_stemmed = stem_words(text1_head_tokenized)
text1_stemmed

'doc docno 105 moseleybraun 19981009 docno text moseleybraun presid want note disappoint perman relief haitian refuge mani other bodi work make law drop treasuri appropri confer report effort began last year debat appropri bill includ languag grant certain central american access suspens deport procedur haitian grant access may recal support grant relief affect class central american along sever colleagu senat hous fought vigor addit provis haitian refuge although unsuccess effort later introduc 1504 haitian immigr fair act 1997 legisl would provid haitian refuge perman resid statu cours'

##### Lemmatizing the words in the tokenized and stemmed text
[Lemmatisation](https://en.wikipedia.org/wiki/Lemmatisation) in linguistics is the process of grouping together the inflected forms of a word so they can be analysed as a single item, identified by the word's lemma, or dictionary form.

In [10]:
def lemmatize_words(text):
    # Creating a lemmatizer instance:
    lemmatizer = WordNetLemmatizer()
    # Applying the lemmatizer word by word:
    lemmatized_words = [lemmatizer.lemmatize(word) for word in word_tokenize(text)]
    # Joining the words back and returning:
    return ' '.join(lemmatized_words)

In [11]:
# Testing the lemmatizer function:
text1_lemmatized = lemmatize_words(text1_stemmed)
text1_lemmatized

'doc docno 105 moseleybraun 19981009 docno text moseleybraun presid want note disappoint perman relief haitian refuge mani other bodi work make law drop treasuri appropri confer report effort began last year debat appropri bill includ languag grant certain central american access suspens deport procedur haitian grant access may recal support grant relief affect class central american along sever colleagu senat hous fought vigor addit provis haitian refuge although unsuccess effort later introduc 1504 haitian immigr fair act 1997 legisl would provid haitian refuge perman resid statu cours'

#### Putting it all together
Now that all the pre-processing functions are tested and working, we can apply the functions to full bodies of both texts.


In [12]:
def text_processor(text):
    step1 = text_preprocessor(text)
    step2 = stem_words(step1)
    step3 = lemmatize_words(step2)
    output = step3
    return output

In [13]:
text1_processed, text2_processed = text_processor(text1), text_processor(text2)

In [14]:
text1_processed[:500]

'doc docno 105 moseleybraun 19981009 docno text moseleybraun presid want note disappoint perman relief haitian refuge mani other bodi work make law drop treasuri appropri confer report effort began last year debat appropri bill includ languag grant certain central american access suspens deport procedur haitian grant access may recal support grant relief affect class central american along sever colleagu senat hous fought vigor addit provis haitian refuge although unsuccess effort later introduc '

#### We represent the processed bodies of text as vectors to analyze them. We use both TF-IDF and Bag-of-Words (Count) approaches.

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Calling an instance of TF-IDF Vectorizer with default arguments:
tfidf_vectorizer = TfidfVectorizer()

# Calling an instance of Count Vectorizer with default arguments:
count_vectorizer = CountVectorizer()

In [16]:
# Vectorizing the bodies of texts and putting them together in a matrix:
corpus_tfidf = tfidf_vectorizer.fit_transform([text1_processed, text2_processed])
corpus_count = count_vectorizer.fit_transform([text1_processed, text2_processed])


In [17]:
import pandas as pd

# Transforming the corpus matrix to a dataframe with feature names (words) as index:
corpus_tfidf_matrix = pd.DataFrame(corpus_tfidf.toarray().transpose(), 
                             index=tfidf_vectorizer.get_feature_names_out())

corpus_count_matrix = pd.DataFrame(corpus_count.toarray().transpose(), 
                             index=count_vectorizer.get_feature_names_out())

# Renaming the columns with the names of the senators who gave the speeches:
corpus_tfidf_matrix = corpus_tfidf_matrix.set_axis(["Moseley-Braun","Reid"], 
                                       axis = "columns", 
                                       copy = True)

corpus_count_matrix = corpus_count_matrix.set_axis(["Moseley-Braun","Reid"], 
                                       axis = "columns", 
                                       copy = True)

In [18]:
corpus_count_matrix

,Moseley-Braun,Reid
000,192,187
060,0,1
063,0,1
083,0,1
097,1,0
...,...,...
zero,13,3
zest,1,0
zombi,1,0
zone,15,1


In [19]:
corpus_tfidf_matrix

,Moseley-Braun,Reid
000,0.047383,0.041278
060,0.000000,0.000310
063,0.000000,0.000310
083,0.000000,0.000310
097,0.000347,0.000000
...,...,...
zero,0.003208,0.000662
zest,0.000347,0.000000
zombi,0.000347,0.000000
zone,0.003702,0.000221


In [20]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculating the cosine similarity between to the vectorized texts:
tfidf_result = cosine_similarity(corpus_tfidf_matrix[corpus_tfidf_matrix.columns[0]].values.reshape(1, -1), 
                  corpus_tfidf_matrix[corpus_tfidf_matrix.columns[1]].values.reshape(1, -1))

bow_result = cosine_similarity(corpus_count_matrix[corpus_count_matrix.columns[0]].values.reshape(1, -1), 
                  corpus_count_matrix[corpus_count_matrix.columns[1]].values.reshape(1, -1))

In [21]:
print("Similarity rate with TF-IDF:", tfidf_result, 
      "\nSimilarity rate with Bag-of-Words:", bow_result)

Similarity rate with TF-IDF: [[0.69861398]] 
Similarity rate with Bag-of-Words: [[0.7534336]]


#### Now let us see how the results change if we look at 2-grams and 3-grams cumulatively in addition to single words.

We need to modify the vectorizers in order to achieve this. Previously, we used the vectorizers with default parameters. This means that they only looked at single words instead of groups of two or three consecutive words.

In [22]:
# Calling an instance of TF-IDF Vectorizer with 1 to 2 grams:
tfidf_vectorizer_12 = TfidfVectorizer(ngram_range = (3,3))

# Calling an instance of Count Vectorizer with 1 to 2 grams:
count_vectorizer_12 = CountVectorizer(ngram_range = (3,3))

# I name the vectorizers with the "_12" suffix, indicating the ngram_range parameter values.

Above, we call the vectorizers to look at 1 and 2-grams together. We could also look at 2-grams only instead by setting the ngram_range parameter to (2,2) instead.

In [23]:
# Vectorizing the bodies of texts and putting them together in a matrix:
corpus_tfidf_12 = tfidf_vectorizer_12.fit_transform([text1_processed, text2_processed])
corpus_count_12 = count_vectorizer_12.fit_transform([text1_processed, text2_processed])

In [24]:
# Transforming the corpus matrix to a dataframe with feature names (words) as index:
corpus_tfidf_matrix_12 = pd.DataFrame(corpus_tfidf_12.toarray().transpose(), 
                             index=tfidf_vectorizer_12.get_feature_names_out())

corpus_count_matrix_12 = pd.DataFrame(corpus_count_12.toarray().transpose(), 
                             index=count_vectorizer_12.get_feature_names_out())

# Renaming the columns with the names of the senators who gave the speeches:
corpus_tfidf_matrix_12 = corpus_tfidf_matrix_12.set_axis(["Moseley-Braun","Reid"], 
                                       axis = "columns", 
                                       copy = True)

corpus_count_matrix_12 = corpus_count_matrix_12.set_axis(["Moseley-Braun","Reid"], 
                                       axis = "columns", 
                                       copy = True)

In [25]:
corpus_count_matrix_12

,Moseley-Braun,Reid
000 000 benefit,1,0
000 000 per,1,0
000 000 portion,0,1
000 000 taxpay,1,0
000 000 week,0,2
...,...,...
zone initi cornerston,1,0
zone new enterpris,1,0
zone peac democraci,1,0
zone urban rural,1,0


In [26]:
corpus_tfidf_matrix_12

,Moseley-Braun,Reid
000 000 benefit,0.001185,0.000000
000 000 per,0.001185,0.000000
000 000 portion,0.000000,0.000735
000 000 taxpay,0.001185,0.000000
000 000 week,0.000000,0.001470
...,...,...
zone initi cornerston,0.001185,0.000000
zone new enterpris,0.001185,0.000000
zone peac democraci,0.001185,0.000000
zone urban rural,0.001185,0.000000


Let us define a function for cosine similarity to make life easier in the future.

In [27]:
def cosine_result(matrix):
    result = cosine_similarity(matrix[matrix.columns[0]].values.reshape(1, -1), 
                               matrix[matrix.columns[1]].values.reshape(1, -1))
    return result

tfidf_result_12 = cosine_result(corpus_tfidf_matrix_12)
bow_result_12 = cosine_result(corpus_count_matrix_12)

print("Similarity rate with TF-IDF:", tfidf_result,
      "\nSimilarity rate with Bag-of-Words:", bow_result, 
      "\nSimilarity rate with TF-IDF, 1 to 2-grams:", tfidf_result_12, 
      "\nSimilarity rate with Bag-of-Words, 1 to 2-grams:", bow_result_12)

Similarity rate with TF-IDF: [[0.69861398]] 
Similarity rate with Bag-of-Words: [[0.7534336]] 
Similarity rate with TF-IDF, 1 to 2-grams: [[0.35161355]] 
Similarity rate with Bag-of-Words, 1 to 2-grams: [[0.5146463]]


Let us define another function which takes two processed texts, ngram_range parameters and vectorizer as input and returns cosine similarity between the two texts.

In [28]:
def similarity_pipeline(vectorizer, txt1, txt2, ngram_range = (1,1)):
    
    # Allowing the option to use Tfidf and Count vectorizers:
    if vectorizer == "Count":
        
        # Calling the vectorizer with desired ngram_range values, (1,1) applies if not specified:
        count_vectorizer = CountVectorizer(ngram_range = ngram_range)
        
        corpus_count = count_vectorizer.fit_transform([txt1, txt2])

        # Loading vectorized texts into a matrix:
        corpus_count_matrix = pd.DataFrame(corpus_count.toarray().transpose(), 
                             index=count_vectorizer.get_feature_names_out())
        
        # Renaming the columns:
        corpus_count_matrix = corpus_count_matrix.set_axis(["Moseley-Braun","Reid"], 
                                       axis = "columns", 
                                       copy = True)
        
        # Defining the output:
        output = cosine_result(corpus_count_matrix)
    
    elif vectorizer == "Tfidf":
        # Calling the vectorizer with desired ngram_range values, (1,1) applies if not specified:
        tfidf_vectorizer = TfidfVectorizer(ngram_range = ngram_range)
        
        corpus_tfidf = tfidf_vectorizer.fit_transform([txt1, txt2])

        # Loading vectorized texts into a matrix:
        corpus_tfidf_matrix = pd.DataFrame(corpus_tfidf.toarray().transpose(), 
                             index=tfidf_vectorizer.get_feature_names_out())
        
        # Renaming the columns:
        corpus_tfidf_matrix = corpus_tfidf_matrix.set_axis(["Moseley-Braun","Reid"], 
                                       axis = "columns", 
                                       copy = True)
        
        # Defining the output:
        output = cosine_result(corpus_tfidf_matrix)

    return output

In [29]:
# Testing the function:
similarity_pipeline("Tfidf", text1_processed, text2_processed, ngram_range = (2,2))

array([[0.46161153]])

Now we can check results comparatively, produced from different vectorizers and specifications of n-grams.

In [31]:
ngram_ranges = [(1,1), (1,2), (1,3), (2,2), (3,3)]
vectorizers = ["Tfidf", "Count"]
results = []
for vec in vectorizers:
    for ngram_range in ngram_ranges:
        if vec == "Tfidf":
            score = similarity_pipeline(vec, text1_processed, text2_processed, ngram_range=ngram_range)
            results.append({"Vectorizer": vec, "Ngram Range": ngram_range, "Similarity Score": score[0][0]})
        else:
            score = similarity_pipeline(vec, text1_processed, text2_processed, ngram_range=ngram_range)
            results.append({"Vectorizer": "Bag-of-Words", "Ngram Range": ngram_range, "Similarity Score": score[0][0]})
df = pd.DataFrame(results)
df

,Vectorizer,Ngram Range,Similarity Score
0,Tfidf,"(1, 1)",0.698614
1,Tfidf,"(1, 2)",0.658574
2,Tfidf,"(1, 3)",0.626878
3,Tfidf,"(2, 2)",0.461612
4,Tfidf,"(3, 3)",0.351614
5,Bag-of-Words,"(1, 1)",0.753434
6,Bag-of-Words,"(1, 2)",0.729314
7,Bag-of-Words,"(1, 3)",0.710265
8,Bag-of-Words,"(2, 2)",0.610052
9,Bag-of-Words,"(3, 3)",0.514646


#### Discussion
To qualitatively discuss the similarity between the two senators' speeches some background on who they are is needed.

[**Moseley-Braun**](https://en.wikipedia.org/wiki/Carol_Moseley_Braun) was *the first African-American woman* elected to the U.S. Senate, the first African-American U.S. Senator from the **Democratic Party**, *the first woman to defeat an incumbent U.S. Senator in an election*, and the first female U.S. Senator from Illinois.

[**Harry Mason Reid Jr.**](https://en.wikipedia.org/wiki/Harry_Reid) was an American lawyer and politician who served as a United States senator from Nevada from 1987 to 2017. He *led the Senate **Democratic** Caucus from 2005 to 2017* and was *the Senate Majority Leader from 2007 to 2015*.

Although both senators were from the same party, the dissimilarity between them should most likely to be rooted in their backgrounds and identities they stand for.

Moseley-Braun had been the first to set many milestones while Reid's election and re-ellection has arguably been in smoother conditions. Moseley-Braun is an African-American woman to be the first female U.S. Senator in her state while Reid is from an already well represented identity group - white and male.

The similarity being above 50% might be due to the fact that they are from the same party but the present difference is, at least superficially, because they are vastly different character and from very different states.

For a better grounded analysis, we can look at the similarity measures of multiple pairs of senator speeches from the same and different parties and employ a comparative perspective. This approach can reveal patterns more clearly as to what makes two speeches similar and what having similar speeches tells us about the characteristics of the senators in comparison. Furthermore, different methods of vectorizing speeches and different measures of similarity might give qualitatively different results.

For instance, we found here a similarity of 65%. A good reference point would be the average level of similarity between senators of the two different parties.